In [1]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from datetime import datetime
import datetime as dt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

#For Prediction
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, cross_validation, svm
from sklearn.svm import SVR
from sklearn import metrics

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [163]:
# Read Datasets 
research_data_raw = pd.read_csv('Datasets/bitcoin_research_data_v2.csv')
# Drop unnecessary columns
research_data = research_data_raw.drop('Unnamed: 0', axis=1)
research_data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0


# Implementation 1

In [164]:
# Convert data to milliseconds
research_data['Epoch'] = pd.to_datetime(research_data['Date'] + ' 23:59:59.000')
research_data['Epoch'] = (research_data['Epoch'] - dt.datetime(1970,1,1)).dt.total_seconds().astype('int64')
research_data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized,Epoch
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0,1557619199
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0,1557532799
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0,1557446399
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0,1557359999
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0,1557273599


In [165]:
research_data_imp2 = research_data.copy()
research_data_imp2.sort_values(by='Date', inplace=True, ascending=True) # Sort by date ascending
forecast_out = int(math.ceil(0.0001 * len(research_data_imp2))) # 0.01% of data
#forecast_out = int(math.ceil(0.0025 * len(research_data_imp2))) # 0.05% of data
print(len(research_data_imp2))
print(forecast_out)

1808
1


In [166]:
research_data_imp2['Prediction'] = research_data_imp2['Close'].shift(-forecast_out)
research_data_imp2.tail()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized,Epoch,Prediction
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0,1557273599,5939.6
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0,1557359999,6151.4
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0,1557446399,6337.9
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0,1557532799,6793.5
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0,1557619199,NaN


In [167]:
X = np.array(research_data_imp2.drop(['Prediction', 'Symbol', 'Date', 'Popularity'], 1)) # Create X with numeric columns only
# X = np.array(research_data_imp2['Close'])
# X = research_data_imp2['Close'].to_frame()
X = preprocessing.scale(X)
X_forecast = X[-forecast_out:]
X = X[:-forecast_out]

# research_data_imp2.dropna(inplace=True)

y = np.array(research_data_imp2['Prediction'])
y = y[:-forecast_out]

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [168]:
# NOT USED
# COULD GRAPH LINEAR KERNEL, BUT WOULD NEED TO PASS X WITH ONLY ONE FEATURE SO ITS SIZE MATCHES Y'S SIZE
def predict_prices(X, Y, X_test, X_forecast): 
    svr_lin  = SVR(kernel='linear', C=1e3) # best
    svr_poly = SVR(kernel='poly', C=1e3, degree=2) # too bad
    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1) # good
     
    # Fit regression model
    svr_lin .fit(X, Y)
    svr_poly.fit(X, Y)
    svr_rbf.fit(X, Y)
    
    plt.scatter(X, Y, c='k', label='Whatever')
    plt.plot(X, svr_lin.predict(X), c='g', label='Linear model')
    plt.plot(X, svr_rbf.predict(X), c='r', label='RBF model')    
    plt.plot(X, svr_poly.predict(X), c='b', label='Polynomial model')
    
    plt.xlabel('X Label')
    plt.ylabel('Y Label')
    plt.title('Support Vector Regression')
    plt.legend()
    plt.show()
    
    return svr_rbf.predict(X_forecast)[0], svr_lin.predict(X_forecast)[0], svr_poly.predict(X_forecast)[0]

# predict_prices(X_train, y_train, X_test, X_forecast)

In [169]:
svr_lin = SVR(kernel='linear', C=1e3)
svr_lin.fit(X_train, y_train)

prediction_lin = svr_lin.predict(X_test)

print("The Explained Variance: %.2f" % svr_lin.score(X_test, y_test))  
print("The Mean Absolute Error: %.2f" % metrics.mean_absolute_error(y_test, prediction_lin))  
print("The Median Absolute Error: %.2f" % metrics.median_absolute_error(y_test, prediction_lin)) 
print("The Mean Squared Error: %.2f" % metrics.mean_squared_error(y_test, prediction_lin)) 
print("The Root Mean Squared Error: %.2f" % (np.sqrt(metrics.mean_squared_error(y_test, prediction_lin))))

svr_lin.predict(X_forecast)[0]

The Explained Variance: 1.00
The Mean Absolute Error: 78.35
The Median Absolute Error: 17.15
The Mean Squared Error: 29076.67
The Root Mean Squared Error: 170.52


6768.761583526393

In [149]:
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
svr_poly.fit(X_train, y_train)

prediction_poly = svr_poly.predict(X_test)

print("The Explained Variance: %.2f" % svr_poly.score(X_test, y_test))  
print("The Mean Absolute Error: %.2f" % metrics.mean_absolute_error(y_test, prediction_poly))  
print("The Median Absolute Error: %.2f" % metrics.median_absolute_error(y_test, prediction_poly)) 
print("The Mean Squared Error: %.2f" % metrics.mean_squared_error(y_test, prediction_poly)) 
print("The Root Mean Squared Error: %.2f" % (np.sqrt(metrics.mean_squared_error(y_test, prediction_poly))))

svr_poly.predict(X_forecast)[0]

The Explained Variance: 0.65
The Mean Absolute Error: 1152.01
The Median Absolute Error: 194.60
The Mean Squared Error: 4606339.81
The Root Mean Squared Error: 2146.24


4133.209495380087

In [150]:
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(X_train, y_train)

prediction_rbf = svr_rbf.predict(X_test)

print("The Explained Variance: %.2f" % svr_rbf.score(X_test, y_test))  
print("The Mean Absolute Error: %.2f" % metrics.mean_absolute_error(y_test, prediction_rbf))  
print("The Median Absolute Error: %.2f" % metrics.median_absolute_error(y_test, prediction_rbf)) 
print("The Mean Squared Error: %.2f" % metrics.mean_squared_error(y_test, prediction_rbf)) 
print("The Root Mean Squared Error: %.2f" % (np.sqrt(metrics.mean_squared_error(y_test, prediction_rbf))))

svr_rbf.predict(X_forecast)[0]

The Explained Variance: 0.98
The Mean Absolute Error: 149.49
The Median Absolute Error: 26.88
The Mean Squared Error: 313413.04
The Root Mean Squared Error: 559.83


6748.905153911851